In [1]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import joblib


In [2]:
# -------------------------------
# Dataset paths
# -------------------------------
train_dir = r"C:\Users\kotaa\Downloads\emotion_recognition_1\emotion_recognition\data\train"


In [3]:
from skimage.feature import hog
import cv2

def extract_hog_features(image):
    image = cv2.resize(image, (48, 48))
    # Convert to grayscale if needed
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Use channel_axis=None for grayscale images
    features, _ = hog(image, orientations=9, pixels_per_cell=(8, 8),
                      cells_per_block=(2, 2), block_norm='L2-Hys',
                      visualize=True, channel_axis=None)
    return features


In [4]:
# -------------------------------
# Load Dataset
# -------------------------------
X, y = [], []

for label in os.listdir(train_dir):
    folder_path = os.path.join(train_dir, label)
    if not os.path.isdir(folder_path):
        continue
    for file in os.listdir(folder_path):
        img_path = os.path.join(folder_path, file)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        features = extract_hog_features(img)
        X.append(features)
        y.append(label)

X = np.array(X)
y = LabelEncoder().fit_transform(y)


In [5]:
# -------------------------------
# Train-Test Split
# -------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [6]:
# Normalize features (important for KNN & LR)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [7]:
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Apply PCA
print("Applying PCA...")
pca = PCA(n_components=150, random_state=42)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Models
models = {
    "Linear SVM": LinearSVC(max_iter=5000, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5, n_jobs=-1),
    "Logistic Regression": LogisticRegression(max_iter=1000, solver="saga", n_jobs=-1)
}

# Train & Evaluate
for name, model in models.items():
    print(f"\n🔹 Training {name}...")
    model.fit(X_train_pca, y_train)
    y_pred = model.predict(X_test_pca)
    acc = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {acc:.4f}")
    print(classification_report(y_test, y_pred))


Applying PCA...

🔹 Training Linear SVM...
Linear SVM Accuracy: 0.4216
              precision    recall  f1-score   support

           0       0.36      0.22      0.27       799
           1       1.00      0.01      0.02        87
           2       0.28      0.12      0.17       820
           3       0.50      0.80      0.62      1443
           4       0.38      0.39      0.38       993
           5       0.32      0.26      0.28       966
           6       0.45      0.56      0.50       634

    accuracy                           0.42      5742
   macro avg       0.47      0.34      0.32      5742
weighted avg       0.40      0.42      0.39      5742


🔹 Training Random Forest...
Random Forest Accuracy: 0.4424
              precision    recall  f1-score   support

           0       0.46      0.21      0.29       799
           1       0.95      0.24      0.39        87
           2       0.45      0.25      0.32       820
           3       0.44      0.86      0.58      1443
  

In [8]:
    # Save each model
joblib.dump(model, f"{name.replace(' ', '_')}_emotion.pkl")

print("\n✅ All models trained and saved!")


✅ All models trained and saved!


In [9]:
# ---------------------------
# Step 3: Train, Evaluate, and Save Models
# ---------------------------
for name, model in models.items():
    print(f"\n🔹 Training {name}...")
    model.fit(X_train_pca, y_train)
    y_pred = model.predict(X_test_pca)
    acc = accuracy_score(y_test, y_pred)

    print(f"{name} Accuracy: {acc:.4f}")
    print(classification_report(y_test, y_pred))

    # Save model
    filename = f"{name.replace(' ', '_')}_emotion.pkl"
    joblib.dump(model, filename)
    print(f"✅ Saved {name} as {filename}")

print("\n🎉 All models trained and saved successfully!")


🔹 Training Linear SVM...
Linear SVM Accuracy: 0.4216
              precision    recall  f1-score   support

           0       0.36      0.22      0.27       799
           1       1.00      0.01      0.02        87
           2       0.28      0.12      0.17       820
           3       0.50      0.80      0.62      1443
           4       0.38      0.39      0.38       993
           5       0.32      0.26      0.28       966
           6       0.45      0.56      0.50       634

    accuracy                           0.42      5742
   macro avg       0.47      0.34      0.32      5742
weighted avg       0.40      0.42      0.39      5742

✅ Saved Linear SVM as Linear_SVM_emotion.pkl

🔹 Training Random Forest...
Random Forest Accuracy: 0.4424
              precision    recall  f1-score   support

           0       0.46      0.21      0.29       799
           1       0.95      0.24      0.39        87
           2       0.45      0.25      0.32       820
           3       0.44    

In [10]:
import joblib
from sklearn.decomposition import PCA

# Example during training
pca = PCA(n_components=100)  
X_train_pca = pca.fit_transform(X_train)

# Save PCA for reuse in testing
joblib.dump(pca, "pca_transform.pkl")


['pca_transform.pkl']